In [ ]:
# 2 quick notes:  Data Leakage & Bayesian Optimization

In [1]:
!pip install bayesian-optimization scikit-learn pandas

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from bayes_opt import BayesianOptimization

In [3]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
X = data.drop('quality', axis=1)
y = data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X.shape

(1599, 11)

In [6]:
def objective(n_estimators, max_depth, min_samples_split, max_features):
    model = RandomForestRegressor(n_estimators=int(n_estimators),
                                  max_depth=int(max_depth),
                                  min_samples_split=int(min_samples_split),
                                  max_features=min(max_features, 0.999),  # Fraction, must be <= 1.0
                                  random_state=42)

    return -1.0 * cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error").mean()

In [7]:
# Bounds for hyperparameters
param_bounds = {
    'n_estimators': (10, 250),
    'max_depth': (1, 50),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.999),
}

In [8]:
optimizer = BayesianOptimization(f=objective, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.3948    | 19.35     | 0.9547    | 18.84     | 153.7     |
| 2         | 0.3985    | 8.645     | 0.2402    | 3.336     | 217.9     |
| 3         | 0.3797    | 30.45     | 0.7366    | 2.473     | 242.8     |
| 4         | 0.3808    | 41.79     | 0.2909    | 6.182     | 54.02     |
| 5         | 0.3886    | 15.91     | 0.5718    | 11.93     | 79.89     |
| 6         | 0.3925    | 8.633     | 0.3423    | 4.297     | 217.6     |
| 7         | 0.4288    | 7.583     | 0.1053    | 2.683     | 218.2     |
| 8         | 0.3924    | 8.95      | 0.6261    | 2.061     | 220.2     |
| 9         | 0.4026    | 6.936     | 0.552     | 2.913     | 219.2     |
| 10        | 0.3996    | 7.72      | 0.4593    | 3.708     | 219.0     |
| 11        | 0.4016    | 6.011     | 0.5897    | 3.48      | 218.0     |
| 12        | 0.4126    | 5.478     | 

In [10]:
best_params = optimizer.max['params']
best_params

{'max_depth': 4.723019905906878,
 'max_features': 0.26880661490436264,
 'min_samples_split': 4.498807222593216,
 'n_estimators': 216.96593364733022}

In [12]:
best_params_formatted = {
    'n_estimators': int(best_params['n_estimators']),
    'max_depth': int(best_params['max_depth']),
    'min_samples_split': int(best_params['min_samples_split']),
    'max_features': best_params['max_features']
}
best_params_formatted

{'n_estimators': 216,
 'max_depth': 4,
 'min_samples_split': 4,
 'max_features': 0.26880661490436264}

In [13]:
optimized_rf = RandomForestRegressor(**best_params_formatted, random_state=42)


In [14]:
optimized_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, max_features=0.26880661490436264,
                      min_samples_split=4, n_estimators=216, random_state=42)

In [15]:
score = optimized_rf.score(X_test, y_test)
print(f"Test R^2 Score with Optimized Hyperparameters: {score}")

Test R^2 Score with Optimized Hyperparameters: 0.377858715533007
